In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [3]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 50, bias= True),
            nn.ReLU(),
            nn.Linear(50, 50, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(50, 45, bias = True),
            nn.ReLU(),
            nn.Linear(45, 40, bias= True),
            nn.Sigmoid(),
            nn.Linear(40, 40, bias= True),
            nn.Sigmoid(),
            nn.Dropout(p= p2),
            nn.Linear(40, 30, bias= True),
            nn.Sigmoid(),
            nn.Linear(30, 25, bias= True),
            nn.Sigmoid(),
            nn.Linear(25, 20, bias= True),
            nn.Sigmoid(),
            nn.Linear(20, 15, bias= True),
            nn.Sigmoid(),
            nn.Linear(15, 10, bias = True),
            nn.ReLU(),
            nn.Linear(10, 5, bias = True),
            nn.ReLU()
        )
        self.classify = nn.Sequential(
            nn.Linear(5, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [4]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [5]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=100, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 100)

In [6]:
model = Network(0.3, 0.6)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=50, out_features=45, bias=True)
    (6): ReLU()
    (7): Linear(in_features=45, out_features=40, bias=True)
    (8): Sigmoid()
    (9): Linear(in_features=40, out_features=40, bias=True)
    (10): Sigmoid()
    (11): Dropout(p=0.6, inplace=False)
    (12): Linear(in_features=40, out_features=30, bias=True)
    (13): Sigmoid()
    (14): Linear(in_features=30, out_features=25, bias=True)
    (15): Sigmoid()
    (16): Linear(in_features=25, out_features=20, bias=True)
    (17): Sigmoid()
    (18): Linear(in_features=20, out_features=15, bias=True)
    (19): Sigmoid()
    (20): Linear(in_features=15, out_features=10, bias=True)
    (21): ReLU()
    (22): Linear(in_features=10, out_features=5, bias=True)
    (23): ReLU()
  )
  (classify): Se

In [7]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001, weight_decay= 0.00001)

In [8]:
for epoch in range(500):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 3355.248402059078
loss - 3338.4548465013504
loss - 3338.4741723537445
loss - 3338.4853679537773
loss - 3338.4663470983505
loss - 3338.475383102894
loss - 3338.4505479335785
loss - 3338.487676501274
loss - 3338.4568335413933
loss - 3338.4292715787888
loss - 3338.4705654382706
loss - 3338.477103292942
loss - 3338.4595419168472
loss - 3338.457520067692
loss - 3338.4772418141365
loss - 3338.4736258387566
loss - 3338.45359647274
loss - 3338.4330908060074
loss - 3338.4662379026413
loss - 3338.469562768936
loss - 3338.4436337947845
loss - 3338.4419174194336
loss - 3338.4545649290085
loss - 3338.4704152941704
loss - 3338.441410958767
loss - 3338.4762080311775
loss - 3338.474545478821
loss - 3338.4597098231316
loss - 3338.459169447422
loss - 3338.4485228657722
loss - 3338.468313395977
loss - 3338.43308699131
loss - 3338.449570477009
loss - 3338.435056209564
loss - 3338.465665638447
loss - 3338.4342972636223
loss - 3338.42248994112
loss - 3338.4530072808266
loss - 3338.4800904393196
loss 

KeyboardInterrupt: 

In [ ]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6132886675504281
